In [2]:
import pygame


class Settings:
    def __init__(self):
        self.window_dimensions = [1200, 800]
        self.screen_fill = [255, 255, 255]

        self.ship_speed = 5

        self.bullet_size = [2, 10]
        self.bullet_speed = 5
        self.bullet_color = [0, 0, 0]

        self.alien_speed = 3
        self.alien_direction = 1
        self.drop_amount = 10

        self.speedup = 1.1
        self.score_adder = 50
        self.score_scaling = 1.5

    def speed_up(self):
        self.ship_speed *= self.speedup
        self.bullet_speed *= self.speedup
        self.alien_speed *= self.speedup
        self.score_adder *= self.score_scaling


class Stats:
    def __init__(self, screen: pygame.Surface, settings: Settings):
        self.hp = 3
        self.score = 0
        self.high_score = 0

        self.text_color = (30, 30, 30)
        self.font = pygame.font.SysFont(None, 48)

        self.screen = screen
        self.settings = settings

    def draw_score(self):
        # convert score and highscore to str
        score_str = str(int(round(self.score, -1)))
        hi_score_str = "High score: " + str(int(round(self.high_score, -1)))
        
        # turning score into an image and drawing it
        score_img = self.font.render(score_str, True, self.text_color, self.settings.screen_fill)
        score_rect = score_img.get_rect()
        score_rect.right = self.screen.get_rect().right - 20
        score_rect.top = 20
        self.screen.blit(score_img, score_rect)

        # turning highscore into an image and drawing it
        score_img = self.font.render(hi_score_str, True, self.text_color, self.settings.screen_fill)
        score_rect = score_img.get_rect()
        score_rect.right = self.screen.get_rect().right - 20
        score_rect.top = 80
        self.screen.blit(score_img, score_rect)


class Keys:
    def __init__(self):
        self.left = False
        self.right = False
        self.z = False


class Ship(pygame.sprite.Sprite):
    def __init__(self, keys: Keys, settings: Settings, screen: pygame.Surface):
        super().__init__()
        self.keys = keys
        self.movement = 0
        self.settings = settings
        self.screen = screen

        # loading ship and placing it at the bottom center
        self.image = pygame.image.load('ship.png')
        self.rect = self.image.get_rect()
        self.rect.midbottom = self.screen.get_rect().midbottom

    def update(self):
        self.movement = 0
        if self.keys.left and self.rect.left > 0:
            self.movement -= self.settings.ship_speed
        if self.keys.right and self.rect.right < self.screen.get_rect().right:
            self.movement += self.settings.ship_speed
        self.rect.centerx += self.movement
        self.draw()

    def draw(self):
        self.screen.blit(self.image, self.rect)


class Bullet(pygame.sprite.Sprite):
    def __init__(self, keys: Keys, settings: Settings, screen: pygame.Surface, ship: Ship):
        super().__init__()
        self.keys = keys
        self.screen = screen
        self.size = settings.bullet_size
        self.color = settings.bullet_color
        self.speed = settings.bullet_speed

        # loading bullet at ship midtop
        self.rect = pygame.Rect([0, 0], self.size)
        self.rect.midtop = ship.rect.midtop

    def update(self):
        self.rect.y -= self.speed
        self.draw()

    def draw(self):
        pygame.draw.rect(self.screen, self.color, self.rect)


class Alien(pygame.sprite.Sprite):
    def __init__(self, settings: Settings, screen: pygame.Surface):
        super().__init__()
        self.settings = settings
        self.screen = screen

        # loading alien and placing it at the bottom center
        self.image = pygame.image.load('alien.png')
        self.rect = self.image.get_rect()

        # placing aLien near top left
        self.rect.topleft = self.rect.size

    def update(self):
        self.movement = self.settings.alien_speed * self.settings.alien_direction
        self.rect.centerx += self.movement
        self.draw()

    def edge(self):
        if self.rect.left <= 0 or self.rect.right >= self.screen.get_rect().right:
            return True

    def draw(self):
        self.screen.blit(self.image, self.rect)


# game functions (general)
def sys_events(event):
    global end, close_game
    if event.type == pygame.QUIT:
        end = True
        close_game = True
    elif event.type == pygame.KEYDOWN:
        keypress(event.key, True)
    elif event.type == pygame.KEYUP:
        keypress(event.key, False)


def keypress(key, toggle_to):
    global keys, bullets
    if key == pygame.K_RIGHT:
        keys.right = toggle_to
    elif key == pygame.K_LEFT:
        keys.left = toggle_to
    elif key == pygame.K_z:
        if toggle_to:
            fire_bullet()


# bullet functions
def fire_bullet():
    global keys, settings, screen, ship
    bullets.add(Bullet(keys, settings, screen, ship))


def update_bullets():
    global bullets
    bullets.update()
    for b in bullets.copy():
        if b.rect.bottom <= 0:
            bullets.remove(b)


# alien functions
def create_alien(column, row):
    global screen, settings, aliens
    alien = Alien(settings, screen)
    alien_width = alien.rect.width
    alien.x = alien_width + 2 * alien_width * column
    alien.rect.x = alien.x
    alien.rect.y = alien.rect.height + 2 * alien.rect.height * row
    aliens.add(alien)


def columns(width):
    return int(settings.window_dimensions[0] / (2 * width)) - 1


def rows(height, ship_height):
    global settings
    return int((settings.window_dimensions[1] - (3 * ship_height)) / (2 * height)) - 1


def new_wave():
    global settings, screen, ship
    temp_alien = Alien(settings, screen)
    no_columns = columns(temp_alien.rect.width)
    no_rows = rows(temp_alien.rect.height, ship.rect.height)

    for row in range(no_rows):
        for column in range(no_columns):
            create_alien(column, row)


def go_down():
    global settings
    for alien in aliens.sprites():
        if alien.edge():
            settings.alien_direction *= -1
            for alien in aliens.sprites():
                alien.rect.y += settings.drop_amount
            break


def update_aliens():
    global aliens
    go_down()
    aliens.update()


# collisions and stuff
def collision():
    global aliens, bullets
    collisions = pygame.sprite.groupcollide(bullets, aliens, True, True)
    if collisions:
        for deads in collisions.values():
            stats.score += settings.score_adder * len(deads)


def dead():
    global stats, end, ship, screen
    if stats.hp > 0:
        stats.hp -= 1

        # clear the screen
        bullets.empty()
        aliens.empty()

        # center the ship
        ship.rect.midbottom = screen.get_rect().midbottom

    else:
        end = True


def check_hit():
    global screen, aliens
    for alien in aliens.sprites():
        if alien.rect.bottom >= screen.get_rect().bottom:
            dead()
            break


def wave_end():
    global aliens
    if len(aliens) == 0:
        new_wave()
        settings.speed_up()
        bullets.empty()


# draw misc.
def draw_hp():
    global stats, screen
    for i in range(stats.hp):
        img = pygame.image.load('ship.png')
        screen.blit(img, (i * img.get_rect().width, 0))

In [3]:
import pygame

settings = Settings()
keys = Keys()
clock = pygame.time.Clock()
pygame.init()
screen = pygame.display.set_mode(settings.window_dimensions)
stats = Stats(screen, settings)
ship = Ship(keys, settings, screen)
bullets = pygame.sprite.Group()
aliens = pygame.sprite.Group()
pygame.display.set_caption("balls")
end = False
close_game = False


while not close_game:
    while not end:
        for e in pygame.event.get():
            sys_events(e)
        screen.fill(settings.screen_fill)
        clock.tick(60)

        ship.update()
        draw_hp()
        stats.draw_score()
        update_bullets()
        update_aliens()
        collision()
        check_hit()
        wave_end()
        pygame.display.update()
    pygame.display.quit()
